In [1]:
# used during development to releoad modules every time there is a change
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from risk_kit import Metrics, pre_processing_ind, get_ind30_nfirms, get_ind30_size, plot_ef2
import ipywidgets as widgets
from IPython.display import display

pd.options.display.float_format = '{:.6f}'.format

## Present Value of Liabilities and funding Ratio
#### The goal of investing is not to grow the assets as large as possibile but it to make sure that you will have the money that you need to do the things that you want to do in the future.

## Objective: setup some prep work to measure two things:
#### 1) Measure the future liability (this future consumption that we need to do the things we want to do)
#### 2) Do I have enough money today to meet my future requirements (point 1) ?

$$ PV(L) = \sum_{i=1}^{k} B(t_i) L_i$$
where $B(t_i)$ is the price of a pure discount bond (Zero Coupon Bond) that pays 1 dollat at time $t_i$
If we assume the yield curve flat and the annual rate of interest is $r$ then $B(t)$ is given by
$$B(t) = \frac{1}{(1+r)^t}$$

In [2]:
m = Metrics()

In [3]:
def discount(t, r):
    """
    Computes the price of a pure discount bond (ZCB) thta pays 1 dollar at time t,
    given the interest rate r
    """
    return 1/(1+r)**t

In [4]:
discount(10, 0.03)

0.7440939148967249

In [5]:
def pv(l,r):
    """
    Computes the present value of a sequence of liabilities
    l is index by the time, and the values are the amounts of each liability
    returns the present value of the sequence
    """
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

In [6]:
# payment schedule: sequence of liabilities: in 3 years I have to pay 1 Million, ecc...
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5])
liabilities

3.000000   1.000000
3.500000   1.500000
4.000000   2.000000
4.500000   2.500000
dtype: float64

In [7]:
# the present value of the sum of the liabilities is less than 7 million
pv(liabilities, 0.03)

6.233320315080044

In [8]:
# in this case we are under funded
m.funding_ratio(5, liabilities, 0.03)

0.8021407126958778

In [9]:
# if interest rates go down the situation is even worse
m.funding_ratio(5, liabilities, 0.02)

0.7720304366941647

In [10]:
# if interest rates go up the situation is better
m.funding_ratio(5, liabilities, 0.05)

0.8649082768407929

In [11]:
def show_funding_ratio(assets, r):
    fr = m.funding_ratio(assets, liabilities, r)
    print(f"{fr*100:.2f}%")

In [12]:
controls = widgets.interactive(show_funding_ratio,
                                assets = widgets.IntSlider(min=1, max=10, step=1, value=5),
                                r = (0, .2, .01)
                              )
display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

COMMENT: if your funding ratio is < 1:
1) you can try to get more assets (in pension funds they ask for more money)
2) You have to find some way to improve the rate of return